In [1]:
import pandas as pd
import numpy as np
import pylab as plt
import matplotlib.pyplot as plt
##import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns
from scipy import stats
color = sns.color_palette()
from sklearn import model_selection, preprocessing
import datetime
import scipy as sp
# Get ready for lots of annoying deprecation warnings
import statsmodels.api as sm
%matplotlib inline

/Users/whalejasmine/anaconda/envs/python2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
